# Implementação do Algoritmo Simplex

### Importação de bibliotecas

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt

### Inicialização do problema

In [4]:
c = [1, 1, 1, 0]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [1, 2, 3, 0],  
    [-1, 2, 6, 0],   
    [0, 4, 9, 0],   
    [0, 0, 3, 1]   
]

# RHS values
b = [3, 2, 5, 1]



In [29]:
import numpy as np
import math

#----// Functions for Printing //----------------

def print_tableau(tableau):
    rows = len(tableau)
    cols = len(tableau[0])
    
    header = ['x' + str(i+1) for i in range(cols-1)] + ['RHS']
    print('    ' + ''.join(f'{col:>8}' for col in header))
    print('    ' + '-' * (8 * cols))
    
    for i in range(rows-1):
        print(f'R{i+1}: ' + ''.join(f'{val:8.2f}' if val != 0 else f'{val:8.0f}' for val in tableau[i]))
    
    print(' z : ' + ''.join(f'{val:8.2f}' if val != 0 else f'{val:8.0f}' for val in tableau[-1]))
    print()

#----// Basic Functions for Tableau //----------------

def eh_basica(coluna):
    # Checks if a column is a basic variable (i.e., forms an identity column)
    return list(coluna).count(1) == 1 and list(coluna).count(0) == len(coluna) - 1

def identify_basic_variables(A):
    A = np.array(A)
    basic_vars = []
    num_rows, num_cols = A.shape
    for j in range(num_cols):
        col = A[:, j]
        if eh_basica(col):
            basic_vars.append(j)
    return basic_vars

#----// Phase 1 //----------------

def fase1(c, A, b):
    # Multiply rows by -1 if RHS is negative
    for i in range(len(b)):
        if b[i] < 0:
            A[i] = [-a for a in A[i]]
            b[i] = -b[i]
    
    num_rows = len(A)
    num_cols = len(A[0])
    
    artificial_vars_indices = []
    
    # Introduce slack and artificial variables
    for i in range(num_rows):
        # Add slack variable
        slack = [0] * num_rows
        slack[i] = 1
        for j in range(num_rows):
            A[j].append(slack[j])
        c.append(0)
        
    num_cols = len(A[0])
    
    # Check for rows that do not have a basic variable
    basic_vars = identify_basic_variables(A)
    for i in range(num_rows):
        row = [A[i][j] for j in range(num_cols)]
        if not any(j in basic_vars and row[j] == 1 for j in range(num_cols)):
            # Add artificial variable
            artificial = [0] * num_rows
            artificial[i] = 1
            for j in range(num_rows):
                A[j].append(artificial[j])
            c.append(1)  # Coefficient of artificial variable in Phase I
            artificial_vars_indices.append(len(c) - 1)
    
    return c, A, b, artificial_vars_indices

#----// Pre-Simplex //----------------

def calcular_z(c, tableau, basic_vars):
    num_rows = len(tableau) - 1  # Exclude the z-row
    num_cols = len(tableau[0])
    
    # Ensure c matches the tableau size
    c_extended = c + [0] * (num_cols - len(c))
    
    # Initialize z-row with zeros
    z_row = [0] * num_cols
    
    # Map basic variables to their rows
    basic_var_to_row = {}
    for row_index in range(num_rows):
        for var_index in basic_vars:
            if tableau[row_index][var_index] == 1 and all(tableau[row_index][k] == 0 for k in range(num_cols - 1) if k != var_index):
                basic_var_to_row[var_index] = row_index
                break
    
    # Sum over basic variables
    for basic_var_index, row_index in basic_var_to_row.items():
        basic_var_coeff = c_extended[basic_var_index]
        row = tableau[row_index]
        for j in range(num_cols):
            z_row[j] += basic_var_coeff * row[j]
    
    # Subtract the objective coefficients
    for j in range(num_cols):
        z_row[j] -= c_extended[j]
    
    return z_row

def para_tableau(c, A, b):
    # Create the main part of the tableau
    tableau = []
    for equation, rhs in zip(A, b):
        tableau.append(equation + [rhs])
    
    # Identify basic variables
    basic_vars = identify_basic_variables(A)
    
    # Calculate z-row based on basic variables
    z_row = calcular_z(c, tableau, basic_vars)
    
    # Append z-row to the tableau
    tableau.append(z_row)
    
    return tableau

#----// Simplex //----------------

def pode_melhorar(tableau):
    # Check if the solution is optimal (no negative coefficients in the z-row)
    return any(x < 0 for x in tableau[-1][:-1])

def obter_posicao_pivo(tableau):
    z = tableau[-1]
    # Choose entering variable (most negative coefficient in z-row)
    coluna = min((i for i in range(len(z)-1)), key=lambda i: z[i])
    if z[coluna] >= 0:
        return None  # Solution is optimal
    
    restricoes = []
    for i, row in enumerate(tableau[:-1]):
        el = row[coluna]
        if el > 0:
            restricoes.append((row[-1] / el, i))
    
    if not restricoes:
        raise Exception("Linear program is unbounded.")
    
    linha = min(restricoes)[1]
    return linha, coluna

def passo_pivo(tableau, posicao_pivo):
    i, j = posicao_pivo
    pivot_element = tableau[i][j]
    num_rows = len(tableau)
    num_cols = len(tableau[0])
    
    # Divide the pivot row by the pivot element
    tableau[i] = [x / pivot_element for x in tableau[i]]
    
    # Update the other rows
    for k in range(num_rows):
        if k != i:
            factor = tableau[k][j]
            tableau[k] = [tableau[k][m] - factor * tableau[i][m] for m in range(num_cols)]
    
    return tableau

def obter_solucao(tableau, num_original_vars):
    num_vars = num_original_vars
    num_rows = len(tableau) - 1  # Exclude the z-row
    solutions = [0] * num_vars
    for var_index in range(num_vars):
        column = [tableau[row_index][var_index] for row_index in range(num_rows)]
        if eh_basica(column):
            row_index = column.index(1)
            solutions[var_index] = tableau[row_index][-1]
        else:
            solutions[var_index] = 0
    return solutions

def simplex(c, A, b, num_original_vars, artificial_vars_indices):
    tableau = para_tableau(c, A, b)
    print_tableau(tableau)
    iteration = 0
    while pode_melhorar(tableau):
        iteration += 1
        posicao_pivo = obter_posicao_pivo(tableau)
        if posicao_pivo is None:
            break
        tableau = passo_pivo(tableau, posicao_pivo)
        print(f"Iteration {iteration}:")
        print_tableau(tableau)
    
    # Check for feasibility if there were artificial variables
    if any(tableau[-1][i] != 0 for i in artificial_vars_indices):
        raise Exception("The linear program is infeasible.")
    
    # Remove artificial variables from the tableau
    for index in sorted(artificial_vars_indices, reverse=True):
        for row in tableau:
            del row[index]
    
    # Remove coefficients of artificial variables from c
    for index in sorted(artificial_vars_indices, reverse=True):
        del c[index]
    
    # Recalculate basic variables and z-row
    A = [row[:-1] for row in tableau[:-1]]
    b = [row[-1] for row in tableau[:-1]]
    tableau = para_tableau(c, A, b)
    print("After removing artificial variables and recalculating z-row:")
    print_tableau(tableau)
    
    # Continue with simplex if necessary
    while pode_melhorar(tableau):
        posicao_pivo = obter_posicao_pivo(tableau)
        if posicao_pivo is None:
            break
        tableau = passo_pivo(tableau, posicao_pivo)
        print_tableau(tableau)
    
    # Extract solution
    solution = obter_solucao(tableau, num_original_vars)
    return solution, tableau

#----// Main Code //----------------

# Objective function coefficients (minimization)
c = [1, 1, 1, 0]  # Coefficients for original variables [y₁, y₂, y₃, y₄]

# Constraint matrix
A = [
    [1, 2, 3, 0],  
    [-1, 2, 6, 0],   
    [0, 4, 9, 0],   
    [0, 0, 3, 1]   
]

# RHS values
b = [3, 2, 5, 1]

# Number of original variables before adding slack and artificial variables
num_original_vars = len(c)

# Perform Phase I (if necessary)
c, A, b, artificial_vars_indices = fase1(c, A, b)

# Run Simplex Method
solution, final_tableau = simplex(c, A, b, num_original_vars, artificial_vars_indices)

# Print Final Solution
print('Final Solution:')
for idx, val in enumerate(solution):
    print(f'y{idx + 1} = {val}')
import numpy as np
import math

#----// Functions for Printing //----------------

def print_tableau(tableau):
    rows = len(tableau)
    cols = len(tableau[0])
    
    header = ['x' + str(i+1) for i in range(cols-1)] + ['RHS']
    print('    ' + ''.join(f'{col:>8}' for col in header))
    print('    ' + '-' * (8 * cols))
    
    for i in range(rows-1):
        print(f'R{i+1}: ' + ''.join(f'{val:8.2f}' if val is not None else '   None ' for val in tableau[i]))
    
    print(' z : ' + ''.join(f'{val:8.2f}' if val is not None else '   None ' for val in tableau[-1]))
    print()

#----// Basic Functions for Tableau //----------------

def eh_basica(coluna):
    # Checks if a column is a basic variable (i.e., forms an identity column)
    return list(coluna).count(1) == 1 and list(coluna).count(0) == len(coluna) - 1

def identify_basic_variables(A):
    A = np.array(A)
    basic_vars = []
    num_rows, num_cols = A.shape
    for j in range(num_cols):
        col = A[:, j]
        if eh_basica(col):
            basic_vars.append(j)
    return basic_vars

#----// Phase 1 //----------------

def fase1(c, A, b):
    # Multiply rows by -1 if RHS is negative
    for line in range(len(b)):
        if b[line] < 0:
            A[line] = [-x for x in A[line]]
            b[line] = -b[line]
    
    artificial_vars_indices = []
    num_rows = len(A)
    num_vars = len(A[0])  # Initial number of variables
    
    # Introduce slack and artificial variables
    for i in range(num_rows):
        # Add slack variable
        slack_var_column = [1 if k == i else 0 for k in range(num_rows)]
        for k in range(num_rows):
            A[k].append(slack_var_column[k])
        c.append(0)  # Coefficient for slack variable
        
        num_vars += 1  # Update the number of variables after adding slack variable
        
        # Now check if the new slack variable column is basic
        slack_col = [A[k][num_vars - 1] for k in range(num_rows)]
        if not eh_basica(slack_col):
            # Add artificial variable
            artificial_var_column = [1 if k == i else 0 for k in range(num_rows)]
            for k in range(num_rows):
                A[k].append(artificial_var_column[k])
            c.append(1)  # Coefficient for artificial variable in Phase I
            artificial_vars_indices.append(len(c) - 1)
            num_vars += 1  # Update the number of variables after adding artificial variable
    
    return c, A, b, artificial_vars_indices

#----// Pre-Simplex //----------------

def calcular_z(c, tableau, basic_vars):
    num_rows = len(tableau) - 1  # Exclude the z-row
    num_cols = len(tableau[0])
    
    # Ensure c matches the tableau size
    c_extended = c + [0] * (num_cols - len(c))
    
    # Initialize z-row with zeros
    z_row = [0] * num_cols
    
    # Map basic variables to their rows
    basic_var_to_row = {}
    for row_index in range(num_rows):
        for var_index in basic_vars:
            if tableau[row_index][var_index] == 1 and all(tableau[row_index][k] == 0 for k in range(num_cols - 1) if k != var_index):
                basic_var_to_row[var_index] = row_index
                break
    
    # Sum over basic variables
    for basic_var_index, row_index in basic_var_to_row.items():
        basic_var_coeff = c_extended[basic_var_index]
        row = tableau[row_index]
        for j in range(num_cols):
            z_row[j] += basic_var_coeff * row[j]
    
    # Subtract the objective coefficients
    for j in range(num_cols):
        z_row[j] -= c_extended[j]
    
    return z_row

def para_tableau(c, A, b):
    # Create the main part of the tableau
    tableau = []
    for equation, rhs in zip(A, b):
        tableau.append(equation + [rhs])
    
    # Identify basic variables
    basic_vars = identify_basic_variables(A)
    
    # Calculate z-row based on basic variables
    z_row = calcular_z(c, tableau, basic_vars)
    
    # Append z-row to the tableau
    tableau.append(z_row)
    
    return tableau

#----// Simplex //----------------

def pode_melhorar(tableau):
    # Check if the solution is optimal (no negative coefficients in the z-row)
    return any(x < 0 for x in tableau[-1][:-1])

def obter_posicao_pivo(tableau):
    z = tableau[-1]
    # Choose entering variable (most negative coefficient in z-row)
    coluna = next(i for i, x in enumerate(z[:-1]) if x < 0)
    
    restricoes = []
    for eq in tableau[:-1]:
        el = eq[coluna]
        restricoes.append(math.inf if el <= 0 else eq[-1] / el)

    if all(r == math.inf for r in restricoes):
        raise Exception("The problem is unbounded.")

    linha = restricoes.index(min(restricoes))
    return linha, coluna

def passo_pivo(tableau, posicao_pivo):
    novo_tableau = [[] for eq in tableau]
    
    i, j = posicao_pivo
    valor_pivo = tableau[i][j]
    novo_tableau[i] = [x / valor_pivo for x in tableau[i]]
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplicador = novo_tableau[i]
            coef = tableau[eq_i][j]
            novo_tableau[eq_i] = [a - coef * b for a, b in zip(tableau[eq_i], multiplicador)]
    
    return novo_tableau

def obter_solucao(tableau, num_original_vars):
    num_vars = num_original_vars
    colunas = np.array(tableau).T
    solucoes = [0] * num_vars
    for idx in range(num_vars):
        coluna = colunas[idx]
        if eh_basica(coluna[:-1]):  # Exclude RHS
            indice_um = np.where(coluna[:-1] == 1)[0][0]
            solucoes[idx] = tableau[indice_um][-1]
    return solucoes

def simplex(c, A, b, num_original_vars):
    tableau = para_tableau(c, A, b)
    print_tableau(tableau)
    while pode_melhorar(tableau):
        posicao_pivo = obter_posicao_pivo(tableau)
        tableau = passo_pivo(tableau, posicao_pivo)
        print_tableau(tableau)
    
    # Extract solution
    solution = obter_solucao(tableau, num_original_vars)
    return solution, tableau

#----// Main Code //----------------

# Objective function coefficients (minimization)
c = [1, 1, 1, 0]  # Coefficients for original variables [y₁, y₂, y₃, y₄]

# Constraint matrix
A = [
    [1, 2, 3, 0],  
    [-1, 2, 6, 0],   
    [0, 4, 9, 0],   
    [0, 0, 3, 1]   
]

# RHS values
b = [3, 2, 5, 1]

# Number of original variables before adding slack and artificial variables
num_original_vars = len(c)

# Perform Phase I (if necessary)
c, A, b, artificial_vars_indices = fase1(c, A, b)

# Run Simplex Method
solution, final_tableau = simplex(c, A, b, num_original_vars)

# Print Final Solution
print('Final Solution:')
for idx, val in enumerate(solution):
    print(f'x{idx + 1} = {val}')
import numpy as np
import math


          x1      x2      x3      x4      x5      x6      x7      x8     RHS
    ------------------------------------------------------------------------
R1:     1.00    2.00    3.00       0    1.00       0       0       0    3.00
R2:    -1.00    2.00    6.00       0       0    1.00       0       0    2.00
R3:        0    4.00    9.00       0       0       0    1.00       0    5.00
R4:        0       0    3.00    1.00       0       0       0    1.00    1.00
 z :    -1.00   -1.00   -1.00       0       0       0       0       0       0

Iteration 1:
          x1      x2      x3      x4      x5      x6      x7      x8     RHS
    ------------------------------------------------------------------------
R1:     1.00    2.00    3.00       0    1.00       0       0       0    3.00
R2:        0    4.00    9.00       0    1.00    1.00       0       0    5.00
R3:        0    4.00    9.00       0       0       0    1.00       0    5.00
R4:        0       0    3.00    1.00       0       0       0 

In [30]:
# Objective function coefficients (minimization)
c = [1, 1, 1, 0]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [1, 2, 3, 0],  
    [-1, 2, 6, 0],   
    [0, 4, 9, 0],   
    [0, 0, 3, 1]   
]

# RHS values
b = [3, 2, 5, 1]

# Perform Phase I (if necessary) and Simplex
c, A, b, artificial_vars_indices = fase1(c, A, b)
solucao = simplex(c, A, b)
print('Solução:', solucao)


TypeError: simplex() missing 1 required positional argument: 'num_original_vars'

In [34]:
# Objective function coefficients (c)
c = [-2, -1, 0, 0]  # coefficients for [x₁, x₂, s₁, s₂]

# Constraint coefficients matrix (A)
A = [
    [1, -1, 1, 0],  # x₁ - x₂ + s₁ = 1
    [1,  2, 0, 1]   # x₁ + 2x₂ + s₂ = 7
]

# Right-hand side values (b)
b = [1, 7]

# Number of original variables before adding slack and artificial variables
num_original_vars = len(c)

# Perform Phase I (if necessary)
c, A, b, artificial_vars_indices = fase1(c, A, b)

# Run Simplex Method
solucao = simplex(c, A, b, num_original_vars)

print('solução: ', solucao)

          x1      x2      x3      x4      x5      x6     RHS
    --------------------------------------------------------
R1:     1.00   -1.00    1.00    0.00    1.00    0.00    1.00
R2:     1.00    2.00    0.00    1.00    0.00    1.00    7.00
 z :     2.00    1.00    0.00    0.00    0.00    0.00    0.00

solução:  ([0, 0, 1, 7], [[1, -1, 1, 0, 1, 0, 1], [1, 2, 0, 1, 0, 1, 7], [2, 1, 0, 0, 0, 0, 0]])


## Problema Pratico

In [125]:
solucao = simplex(c, A, b)
print('solução do problema prático: ', solucao)

          x1      x2      x3      x4     RHS
    ----------------------------------------
R1:     1.00   -1.00    1.00    0.00    1.00
R2:     1.00    2.00    0.00    1.00    7.00
z :     2.00    1.00    0.00    0.00    0.00

Coluna Basica: [1 0 0]
Coluna Basica: [0 1 0]
solução do problema prático:  [0, 0, np.int64(1), np.int64(7), 0]



    Implicit Initial Basic Solution
    The initial basic feasible solution is:

    Set all original variables (x₁, x₂) to 0
    Slack variables (x₃, x₄, x₅) take the values of the RHS (b vector)
    This works because the slack variables form an identity matrix in the constraint matrix

Important Note: This code assumes the problem is already in standard form and doesn't implement Phase I of the simplex method. It would need modification to handle:

    Problems with ≥ constraints
    Problems with negative RHS values
    Problems requiring artificial variables

To make this code more robust, you would need to add Phase I implementation to handle cases where an initial basic feasible solution isn't immediately available.


In [32]:
# Objective function coefficients (minimization, so we use negative values)
c = [-3, -4, 0, 0]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [1, 0, 1, 0],  # y₁ + s₁ = 2
    [0, 1, 0, 1]   # y₂ + s₂ = 1
]

# RHS values
b = [2, 1]

solucao = simplex(c, A, b)
print('solução do problema prático: ', solucao)

TypeError: simplex() missing 1 required positional argument: 'num_original_vars'

In [19]:

# Objective function coefficients (minimization, so we use negative values)
c = [1, 1, 1, 0]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [1, 2, 3, 0],  
    [-1, 2, 6, 0],   
    [0, 4, 9, 0],   
    [0, 0, 3, 1]   
]

# RHS values
b = [3, 2, 5, 1]

solucao = simplex(c, A, b)
print('solução do problema prático: ', solucao)

TypeError: simplex() missing 2 required positional arguments: 'num_original_vars' and 'artificial_vars_indices'